<a href="https://colab.research.google.com/github/abdoezzat1323/Bloggie/blob/main/abdelrhman%20ezzat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div class="markdown-google-sans">

## **Getting started**
</div>

Wikipedia Dataset (Simple english) from 🤗 datasets
Install the package
Download the dataset
Convert to Dataframe and save as .csv (comma separated file) for later usage
You may use spark to read it line by line as the wordcounter example and split on tab "\n"

In [3]:
!pip install datasets
!pip install pyspark
!pip install apache_beam

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 14.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.2 MB/s eta 0:

In [4]:
from datasets import load_dataset

dataset = load_dataset("wikipedia", "20220301.simple")

Downloading:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/235M [00:00<?, ?B/s]

Dataset wikipedia downloaded and prepared to /root/.cache/huggingface/datasets/wikipedia/20220301.simple/2.0.0/aa542ed919df55cc5d3347f42dd4521d05ca68751f50dbc32bae2a7f1e167559. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
dataset["train"].select(range(5)).to_pandas()

,id,url,title,text
0,1,https://simple.wikipedia.org/wiki/April,April,April is the fourth month of the year in the J...
1,2,https://simple.wikipedia.org/wiki/August,August,August (Aug.) is the eighth month of the year ...
2,6,https://simple.wikipedia.org/wiki/Art,Art,Art is a creative activity that expresses imag...
3,8,https://simple.wikipedia.org/wiki/A,A,A or a is the first letter of the English alph...
4,9,https://simple.wikipedia.org/wiki/Air,Air,Air refers to the Earth's atmosphere. Air is a...


*italicized text*<div class="markdown-google-sans">

</div>

Generate the dataset
random_state = 100 ensures we get the same subset every time (reproducibility)

In [6]:
dataset["train"].to_pandas()[["title","text","id"]].sample(20000,random_state=100).to_csv("dataset.csv", index=False,compression={'method': 'gzip', 'compresslevel': 3, 'mtime': 1})

In [7]:
!pip install tabulate
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# **code without join query↓↓↓↓↓↓↓↓↓**

---



In [27]:
import time
from collections import Counter
from pyspark import SparkContext

import pandas as pd
from tabulate import tabulate 
start = time.time()

dataset = pd.read_csv("dataset.csv", compression={'method': 'gzip', 'compresslevel': 3, 'mtime': 1})
index = {title: doc_words for title, doc_words in dataset[["title", "text"]].values.tolist()}
dataset["text"] = dataset["text"].str.lower().str.replace('[^\w\s]', '', regex=True).str.split()
dataset = dataset[["title", "text"]].values.tolist()
sc.stop()
sc = SparkContext("local", "MyApp")

dataset_rdd = sc.parallelize(dataset)


words = dataset_rdd.flatMap(lambda pair: [(pair[0], word) for doc_words in pair[1] for word in doc_words])

word_title_pairs = words.map(lambda pair: (pair[1], pair[0]))

inverted_idx = word_title_pairs.groupByKey().mapValues(list).collectAsMap()



query_bow = set("egypt pyramids".lower().split())


word_doc_rdd = dataset_rdd.flatMap(lambda pair: [(word, pair[0]) for word in pair[1]])


word_doc_grouped = word_doc_rdd.groupByKey()


inverted_index = word_doc_grouped.mapValues(list).collectAsMap()


docs_for_query_words = []
for query_word in query_bow:
    docs = word_doc_grouped.filter(lambda pair: pair[0] == query_word).flatMap(lambda pair: pair[1]).collect()
    docs_for_query_words.append(docs)


final_scores = Counter()
for docs_for_query_word in docs_for_query_words:
    docs_for_query_word = set(docs_for_query_word)  
    for doc_name in docs_for_query_word:
        final_scores[doc_name] += 1
table = []

for title, score in final_scores.most_common(10):
    table.append([title, score, index[title].replace("\n", " ")[:160]])

table.sort(key=lambda item: (-item[1], item[0]))
print(tabulate(table, ["title", "score", "paragraph"], tablefmt="fancy_grid"))

print(time.time()-start)

╒════════════════════════════╤═════════╤══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╕
│ title                      │   score │ paragraph                                                                                                                                                        │
╞════════════════════════════╪═════════╪══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╡
│ 30 June Stadium            │       2 │ 30 June Stadium is a stadium in Cairo, Egypt. It has a capacity of 30,000 people. It was built by the Egyptian Air Defense Forces.  The stadium is used by the E │
├────────────────────────────┼─────────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

# code with join **Query ↓↓↓↓↓↓↓**

In [28]:
import time
from collections import Counter
from pyspark import SparkContext

import pandas as pd
from tabulate import tabulate 
start = time.time()

dataset = pd.read_csv("dataset.csv", compression={'method': 'gzip', 'compresslevel': 3, 'mtime': 1})
index = {title: doc_words for title, doc_words in dataset[["title", "text"]].values.tolist()}
dataset["text"] = dataset["text"].str.lower().str.replace('[^\w\s]', '', regex=True).str.split()
dataset = dataset[["title", "text"]].values.tolist()
sc.stop()
sc = SparkContext("local", "MyApp")

dataset_rdd = sc.parallelize(dataset)


words = dataset_rdd.flatMap(lambda pair: [(pair[0], word) for doc_words in pair[1] for word in doc_words])

word_title_pairs = words.map(lambda pair: (pair[1], pair[0]))

inverted_idx = word_title_pairs.groupByKey().mapValues(list).collectAsMap()



query_bow = set("egypt pyramids".lower().split())


word_doc_rdd = dataset_rdd.flatMap(lambda pair: [(word, pair[0]) for word in pair[1]])


word_doc_grouped = word_doc_rdd.groupByKey()


inverted_index = word_doc_grouped.mapValues(list).collectAsMap()


query_rdd = sc.parallelize([(w, None) for w in query_bow])

joined_rdd = word_doc_grouped.join(query_rdd)

docs_for_query_words = joined_rdd.map(lambda pair: pair[1][0]).collect()


final_scores = Counter()
for docs_for_query_word in docs_for_query_words:
    docs_for_query_word = set(docs_for_query_word)  
    for doc_name in docs_for_query_word:
        final_scores[doc_name] += 1
table = []

for title, score in final_scores.most_common(10):
    table.append([title, score, index[title].replace("\n", " ")[:160]])

table.sort(key=lambda item: (-item[1], item[0]))
print(tabulate(table, ["title", "score", "paragraph"], tablefmt="fancy_grid"))

print(time.time()-start)

╒════════════════════════════╤═════════╤══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╕
│ title                      │   score │ paragraph                                                                                                                                                        │
╞════════════════════════════╪═════════╪══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╡
│ 30 June Stadium            │       2 │ 30 June Stadium is a stadium in Cairo, Egypt. It has a capacity of 30,000 people. It was built by the Egyptian Air Defense Forces.  The stadium is used by the E │
├────────────────────────────┼─────────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────